# Stress Region Predictor: Mining & Retraining Pipeline

This notebook handles the end-to-end process of creating a custom stress-response dataset and retraining the predictor models.

**Pipeline Overview:**
1. **Target Identification**: search NCBI for genes related to specific stress keywords (e.g., "drought", "salt stress").
2. **Data Mining**: Download promoter/gene sequences for these targets and scan them for known PLACE motifs.
3. **Model Training**: 
    - **ML Benchmark**: Compares SVM, Random Forest, etc., to find the best statistical model.
    - **Deep Learning**: Fine-tunes PlantBERT on the new specific dataset.
4. **Evaluation**: Ensembles the models and evaluates performance on a hold-out test set.

---

## 1. Target Identification (NCBI Search)
*Searches the NCBI Gene database for Arabidopsis thaliana genes associated with stress keywords.*

In [3]:
from Bio import Entrez
import time
import re

# =============================================================================
# KONFIGURASI USER
# =============================================================================
Entrez.email = "email_anda@example.com"  # Ganti dengan email aktif
organism = "Arabidopsis thaliana"

# List keyword sama seperti sebelumnya
stress_keywords = [
    "drought", "salt stress", "cold stress", "heat shock", 
    "abscisic acid", "water deprivation", "oxidative stress", "salinity"
]

output_file = "list_gen_stres.txt"

# =============================================================================
# FUNGSI PENCARIAN & FILTER AKSESI
# =============================================================================
def fetch_gene_accessions():
    print(f"--- MEMULAI PENCARIAN GEN STRES (MODE AKSESI) ---")
    
    # Query Search
    term_query = f'"{organism}"[Orgn] AND ({ " OR ".join(stress_keywords) })'
    
    # Regex untuk mendeteksi Locus Tag Arabidopsis (contoh: AT1G01010)
    # AT = Arabidopsis, [1-5MC] = Kromosom 1-5/Mito/Chloro, G = Gen, \d{5} = 5 digit angka
    locus_tag_pattern = re.compile(r'AT[1-5MC]G\d{5}', re.IGNORECASE)

    try:
        # 1. Search ke Database NCBI Gene
        print(f"\n[1/3] Mengirim request ke NCBI...")
        search_handle = Entrez.esearch(db="gene", term=term_query, retmax=10000, usehistory="y")
        search_results = Entrez.read(search_handle)
        search_handle.close()
        
        count = int(search_results["Count"])
        webenv = search_results["WebEnv"]
        query_key = search_results["QueryKey"]
        
        print(f"      -> Ditemukan {count} kandidat gen.")
        
        if count == 0:
            return

        # 2. Fetch Detail & Filter ID
        print(f"[2/3] Memilah ID Aksesi (Locus Tag) yang valid...")
        
        valid_accessions = set()
        batch_size = 300
        
        for start in range(0, count, batch_size):
            fetch_handle = Entrez.esummary(
                db="gene", 
                retstart=start, 
                retmax=batch_size, 
                webenv=webenv, 
                query_key=query_key
            )
            data = Entrez.read(fetch_handle)
            fetch_handle.close()
            
            for doc in data['DocumentSummarySet']['DocumentSummary']:
                # Kumpulkan semua kemungkinan nama dari gen ini
                potential_ids = []
                
                # 1. Cek Nama Utama
                if 'Name' in doc: 
                    potential_ids.append(doc['Name'])
                
                # 2. Cek Nama Lain (OtherAliases) - biasanya Locus Tag sembunyi disini
                if 'OtherAliases' in doc and doc['OtherAliases']:
                    # OtherAliases formatnya string dipisah koma
                    aliases = doc['OtherAliases'].split(',')
                    potential_ids.extend([a.strip() for a in aliases])

                # LOGIKA PEMILIHAN ID:
                # Kita cari ID yang formatnya "ATxGxxxxx". Ini ID paling aman buat download.
                selected_id = None
                
                for pid in potential_ids:
                    if locus_tag_pattern.fullmatch(pid):
                        selected_id = pid.upper()
                        break # Ketemu Locus Tag! Stop cari.
                
                # Jika tidak ada Locus Tag, ambil Name biasa (cadangan)
                if not selected_id and 'Name' in doc:
                    selected_id = doc['Name']
                
                if selected_id:
                    valid_accessions.add(selected_id)
            
            print(f"      -> Progress: {len(valid_accessions)} ID valid terkumpul...", end='\r')
            time.sleep(0.5)

        print(f"\n      -> Selesai! {len(valid_accessions)} ID siap pakai.")

        # 3. Simpan
        with open(output_file, "w") as f:
            for acc in sorted(valid_accessions):
                f.write(acc + "\n")
        
        print(f"\n[SUKSES] File tersimpan: {output_file}")
        print("Sekarang isi file ini adalah ID Locus Tag (misal: AT1G01010) yang pasti bisa didownload.")

    except Exception as e:
        print(f"\n[ERROR] {e}")

if __name__ == "__main__":
    fetch_gene_accessions()

--- MEMULAI PENCARIAN GEN STRES (MODE AKSESI) ---

[1/3] Mengirim request ke NCBI...
      -> Ditemukan 2587 kandidat gen.
[2/3] Memilah ID Aksesi (Locus Tag) yang valid...
      -> Progress: 2585 ID valid terkumpul...
      -> Selesai! 2585 ID siap pakai.

[SUKSES] File tersimpan: list_gen_stres.txt
Sekarang isi file ini adalah ID Locus Tag (misal: AT1G01010) yang pasti bisa didownload.


## 2. Sequence Mining & Motif Scanning
*Downloads the specific sequences for the identified genes and scans them for motifs found in the PLACE database. This creates the "Positive" dataset (Label 1).*

In [5]:
# =============================================================================
# NCBI MINER FINAL: FIX GIANT CHROMOSOME BUG
# =============================================================================
from Bio import Entrez
import pandas as pd
import time
import os
import re

# =============================================================================
# 1. KONFIGURASI
# =============================================================================
Entrez.email = "plantbert_finetune@example.com"

# Filter Motif
motif_filter_keywords = ["ABRE", "DRE", "G-BOX", "MYB", "WRKY", "NAC"]

# Limit Gen (Bisa diisi angka atau None)
LIMIT_GENES = 100 

# PENTING: Batas Maksimal Panjang Sequence (agar tidak download kromosom)
MAX_SEQ_LEN = 20000  # 20kb. Jika lebih dari ini, pasti salah download.

# File Paths
place_csv_path = '/Users/user/Downloads/02. PROJECTS/Stress-region-predictor/train/PLACE_Parsed_Complete_V2.csv'
gene_list_path = 'list_gen_stres.txt' 
output_csv = f'Dataset_Fixed_Top{LIMIT_GENES}.csv'
flank_bp = 50

# =============================================================================
# A. PERSIAPAN MOTIF
# =============================================================================
iu_map = {
    'A': 'A', 'C': 'C', 'G': 'G', 'T': 'T', 'U': 'T',
    'R': '[AG]', 'Y': '[CT]', 'M': '[AC]', 'K': '[GT]', 'S': '[GC]', 'W': '[AT]',
    'H': '[ACT]', 'B': '[CGT]', 'V': '[ACG]', 'D': '[AGT]', 'N': '[ACGT]'
}

def iupac_to_regex_pattern(seq):
    seq = str(seq).upper().strip()
    seq = "".join([c for c in seq if c in iu_map]) 
    res = ""
    for c in seq:
        res += iu_map.get(c, c)
    return res

print(f"--- [1/3] Memuat Library Motif PLACE ---")
motif_library = []

if os.path.exists(place_csv_path):
    df_place = pd.read_csv(place_csv_path)
    col_seq = 'Sequence' if 'Sequence' in df_place.columns else df_place.columns[1]
    col_id = 'ID' if 'ID' in df_place.columns else df_place.columns[0]

    unique_motifs = df_place[[col_id, col_seq]].drop_duplicates()
    
    for idx, row in unique_motifs.iterrows():
        m_id = str(row[col_id]).upper()
        m_seq = row[col_seq]
        
        is_match = False
        for k in motif_filter_keywords:
            if k.upper() in m_id:
                is_match = True
                break
        
        if is_match and len(str(m_seq)) >= 3:
            motif_library.append({
                'id': row[col_id],
                'original': m_seq,
                'regex': re.compile(iupac_to_regex_pattern(m_seq))
            })
    print(f"Total Motif Terpilih: {len(motif_library)}")
    if len(motif_library) == 0: exit()
else:
    print("Error: CSV tidak ditemukan.")
    exit()

# =============================================================================
# B. FUNGSI FETCH (LEBIH KETAT)
# =============================================================================
def fetch_sequence_strict(locus_tag):
    try:
        # PERBAIKAN 1: Tambah 'biomol_mrna[PROP]' agar tidak ambil kromosom
        query = f"{locus_tag}[Gene Name] AND Arabidopsis thaliana[Orgn] AND refseq[filter] AND biomol_mrna[PROP]"
        
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=1)
        record = Entrez.read(handle)
        handle.close()
        
        if not record['IdList']: return None, "No mRNA found"

        seq_id = record['IdList'][0]
        
        with Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text") as handle:
            fasta_data = handle.read()
        
        lines = fasta_data.strip().split('\n')
        if not lines: return None, "Empty FASTA"
        
        full_seq = "".join(lines[1:]).upper()
        
        # PERBAIKAN 2: Cek Ukuran
        if len(full_seq) > MAX_SEQ_LEN:
            return None, f"OVERSIZE ({len(full_seq)} bp) - Skipped"
            
        return full_seq, "Success"

    except Exception as e:
        return None, str(e)

# =============================================================================
# C. MAIN LOOP
# =============================================================================
print("\n--- [2/3] Memuat Daftar Target Gen ---")
target_genes = []
if os.path.exists(gene_list_path):
    with open(gene_list_path, 'r') as f:
        all_genes = [line.strip() for line in f if line.strip()]
    
    if LIMIT_GENES and len(all_genes) > LIMIT_GENES:
        target_genes = all_genes[:LIMIT_GENES]
    else:
        target_genes = all_genes
else:
    exit()

print(f"--- [3/3] Mulai Mining ({len(target_genes)} Gen) ---")
results = []
batch_save = 10 

for i, gid in enumerate(target_genes):
    print(f"[{i+1}/{len(target_genes)}] {gid}: ", end="")
    
    # 1. Fetch Strict
    seq, status = fetch_sequence_strict(gid)
    
    if seq:
        print(f"OK ({len(seq)}bp). Scan...", end=" ")
        
        hits = 0
        for m in motif_library:
            for match in m['regex'].finditer(seq):
                start, end = match.start(), match.end()
                
                if start >= flank_bp and (end + flank_bp) <= len(seq):
                    results.append({
                        'Gene_Locus': gid,
                        'Motif_ID': m['id'],
                        'Motif_Pattern': m['original'],
                        'Extracted_Sequence': seq[start-flank_bp : end+flank_bp],
                        'Label': 1
                    })
                    hits += 1
        print(f"Found {hits}.")
    else:
        # Print error merah/jelas jika oversize atau gagal
        print(f"SKIP -> {status}")
    
    time.sleep(1.0) 

    if len(results) > 0 and i % batch_save == 0:
        pd.DataFrame(results).to_csv(output_csv, index=False)

if results:
    final_df = pd.DataFrame(results).drop_duplicates(subset=['Extracted_Sequence'])
    final_df.to_csv(output_csv, index=False)
    print(f"\n[SELESAI] Data bersih tersimpan: {output_csv}")
    print(f"Total Sequence: {len(final_df)}")
else:
    print("\n[ZONK] Tidak ada data.")

--- [1/3] Memuat Library Motif PLACE ---
Total Motif Terpilih: 47

--- [2/3] Memuat Daftar Target Gen ---
--- [3/3] Mulai Mining (100 Gen) ---
[1/100] AT1G01060: OK (2132bp). Scan... Found 27.
[2/100] AT1G01140: OK (1838bp). Scan... Found 21.
[3/100] AT1G01183: SKIP -> No mRNA found
[4/100] AT1G01230: OK (876bp). Scan... Found 8.
[5/100] AT1G01360: OK (1179bp). Scan... Found 5.
[6/100] AT1G01470: OK (1348bp). Scan... Found 18.
[7/100] AT1G01480: OK (2002bp). Scan... Found 18.
[8/100] AT1G01510: OK (2637bp). Scan... Found 30.
[9/100] AT1G01560: OK (1636bp). Scan... Found 23.
[10/100] AT1G01620: OK (1549bp). Scan... Found 11.
[11/100] AT1G01720: OK (1653bp). Scan... Found 27.
[12/100] AT1G01790: OK (4237bp). Scan... Found 34.
[13/100] AT1G02130: OK (1110bp). Scan... Found 15.
[14/100] AT1G02205: OK (2705bp). Scan... Found 30.
[15/100] AT1G02340: OK (1466bp). Scan... Found 7.
[16/100] AT1G02500: OK (1748bp). Scan... Found 30.
[17/100] AT1G02650: OK (1916bp). Scan... Found 27.
[18/100] AT1

## 3. Model Retraining & Benchmarking
*Trains a suite of machine learning models (SVM, RF, GB, LR) to find the best baseline, then fine-tunes the advanced PlantBERT model on the new data.*

In [7]:
# =============================================================================
# RETRAINING STAGE: TRAIN ALL MODELS (ML BENCHMARK + PLANTBERT)
# =============================================================================
import sys
import os
import importlib

# 1. ADD MODULE PATH
project_root = "/Users/user/Downloads/02. PROJECTS/Stress-region-predictor"
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. FORCE RELOAD & IMPORT FUNCTIONS
# Ideally, you should restart the kernel if code changes significantly, 
# but this helps picking up new functions on the fly.
import train.ensemble_predictor
importlib.reload(train.ensemble_predictor)
from train.ensemble_predictor import train_multimodel_ml, train_plantbert_from_mined_data

# 3. DEFINE INPUT
try:
    # Use variable from previous cell if available
    mined_csv = output_csv
except NameError:
    # Fallback default
    mined_csv = "Dataset_Fixed_Top100.csv"

print(f"Using Mined Dataset: {mined_csv}")

# 4. TRAIN MODELS
print(">>> PART 1: Multi-Model ML Benchmark (Finding the Champion) <<<")
# This will run SVM, RandomForest, GradientBoosting, and LogisticRegression
# It selects the BEST one automatically.
best_ml_model, vectorizer = train_multimodel_ml(mined_csv)

print("\n>>> PART 2: Retraining PlantBERT Model (Deep Learning) <<<")
# This trains the heavy transformer model
plantbert_model, bert_tokenizer = train_plantbert_from_mined_data(mined_csv)


Using Mined Dataset: Dataset_Fixed_Top100.csv
>>> PART 1: Multi-Model ML Benchmark (Finding the Champion) <<<

--- STARTING MULTI-MODEL ML TRAINING ---
      -> Data split: 3126 Train, 782 Test
      -> Feature Engineering (3-5 gram char vectorizer)...

      -> Training LogisticRegression...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


         Best Params: {'C': 0.1, 'solver': 'liblinear'}
         Test Accuracy: 89.39% (1.8s)

      -> Training RandomForest...
         Best Params: {'max_depth': 20, 'n_estimators': 200}
         Test Accuracy: 90.66% (8.6s)

      -> Training SVM...
         Best Params: {'C': 10, 'kernel': 'rbf'}
         Test Accuracy: 93.48% (94.8s)

      -> Training GradientBoosting...
         Best Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
         Test Accuracy: 90.92% (13.0s)

      --- MODEL LEADERBOARD ---
             Model  Accuracy                                                 Best Params
               SVM  0.934783                                  {'C': 10, 'kernel': 'rbf'}
  GradientBoosting  0.909207 {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
      RandomForest  0.906650                      {'max_depth': 20, 'n_estimators': 200}
LogisticRegression  0.893862                           {'C': 0.1, 'solver': 'liblinear'}

      -> 🏆 WINNER: 

Map: 100%|██████████| 782/782 [00:00<00:00, 26239.78 examples/s]


      -> Found potential checkpoint: /Users/user/Downloads/02. PROJECTS/Stress-region-predictor/train/plantbert/model_finetune_expanded/final
      -> Loading Model Weights from: /Users/user/Downloads/02. PROJECTS/Stress-region-predictor/train/plantbert/model_finetune_expanded/final


...

{'eval_loss': 0.29973772168159485, 'eval_accuracy': 0.928388746803069, 'eval_runtime': 9.6649, 'eval_samples_per_second': 80.912, 'eval_steps_per_second': 5.07, 'epoch': 3.0}


100%|██████████| 588/588 [17:46<00:00,  1.81s/it]


{'train_runtime': 1066.4909, 'train_samples_per_second': 8.793, 'train_steps_per_second': 0.551, 'train_loss': 0.1897736503837668, 'epoch': 3.0}
      -> Saving PlantBERT to /Users/user/Downloads/02. PROJECTS/Stress-region-predictor/train/plantbert/model_finetune_expanded/plantbert_finetuned_mined/final


/Users/user/miniforge3/envs/promac/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 49/49 [00:09<00:00,  5.22it/s]


      -> Final Evaluation: {'eval_loss': 0.29973772168159485, 'eval_accuracy': 0.928388746803069, 'eval_runtime': 9.7041, 'eval_samples_per_second': 80.584, 'eval_steps_per_second': 5.049, 'epoch': 3.0}


## 4. Evaluation & Deployment
*Runs the final ensemble model on a hold-out test set to calculate accuracy and classification metrics, then saves the optimal models for future use.*

In [12]:
# =============================================================================
# FINAL STEP: PREDICT & SAVE YOUR NEW MODELS
# =============================================================================
import joblib
import os
import importlib

# 1. RELOAD MODULE TO CATCH UPDATES
# This fixes the ImportError by forcing Python to look at the updated file
import train.ensemble_predictor
importlib.reload(train.ensemble_predictor)
from train.ensemble_predictor import predict_with_new_models, evaluate_models_on_holdout

# 2. RUN & SAVE
if 'best_ml_model' in locals() and 'plantbert_model' in locals():
    # A. Get Accuracy Summary (New Feature)
    print("\n[INFO] Calculating Accuracy on Test Set...")
    metrics = evaluate_models_on_holdout(
        mined_csv, 
        svm_model=best_ml_model, 
        vectorizer=vectorizer, 
        bert_model=plantbert_model, 
        bert_tokenizer=bert_tokenizer
    )

    # B. Run Prediction
    df_results = predict_with_new_models(
        mined_csv, 
        svm_model=best_ml_model, 
        vectorizer=vectorizer, 
        bert_model=plantbert_model, 
        bert_tokenizer=bert_tokenizer
    )
    
    # C. Save Models to Disk for Future Use
    print("\n--- SAVING MODELS ---")
    
    # Save ML Model (e.g. SVM/RF)
    ml_path = "final_best_ml_model.pkl"
    joblib.dump(best_ml_model, ml_path)
    joblib.dump(vectorizer, ml_path.replace(".pkl", "_vectorizer.pkl"))
    print(f"   -> ML Model saved to: {ml_path}")
    
    # Save PlantBERT (Deep Learning)
    bert_path = "final_plantbert_model"
    plantbert_model.save_pretrained(bert_path)
    bert_tokenizer.save_pretrained(bert_path)
    print(f"   -> PlantBERT saved to: {bert_path}/")

    # D. Show Top Hits
    if df_results is not None:
        print("\n--- TOP HIGH CONFIDENCE PREDICTIONS ---")
        print(df_results.sort_values(by="Stress_Probability", ascending=False).head(10)[['Gene_Locus', 'Motif_ID', 'ML_Prob', 'BERT_Prob', 'Stress_Probability']])
else:
    print("Please run the training cell above first!")


[INFO] Calculating Accuracy on Test Set...

--- EVALUATING MODEL ACCURACY ON TEST SET (20% Hold-out) ---
   -> ML Model Accuracy: 94.76%
   -> Evaluating PlantBERT (this takes a moment)...
   -> PlantBERT Accuracy: 92.71%

--- RUNNING FINAL ENSEMBLE PREDICTION ---
      -> getting ML probabilities...
      -> getting PlantBERT probabilities...
      -> Saved to Final_Ensemble_Predictions.csv

--- SAVING MODELS ---
   -> ML Model saved to: final_best_ml_model.pkl
   -> PlantBERT saved to: final_plantbert_model/

--- TOP HIGH CONFIDENCE PREDICTIONS ---
     Gene_Locus         Motif_ID  ML_Prob  BERT_Prob  Stress_Probability
345   AT1G02730          MYBCORE      1.0     0.9997              0.9999
1602  AT1G08430          AMYBOX2      1.0     0.9997              0.9999
842   AT1G04410  MYB2CONSENSUSAT      1.0     0.9997              0.9999
1677  AT1G08800          MYBCORE      1.0     0.9997              0.9999
571   AT1G03060         WRKY71OS      1.0     0.9997              0.9999
371 